In [1]:
import torch
from transformers import AutoModel, AutoTokenizer
from huggingface_hub import notebook_login

import sys
import os
sys.path.append("/data2/scratch/junhalee/extract-llama-embed/")

from src.utils.constants import LLAMA_3P3_70B_MODEL_DIR

In [ ]:
notebook_login()

TypeError: notebook_login() got an unexpected keyword argument 'hf_token'

In [ ]:
model_name = "meta-llama/Llama-3.3-70B-Instruct"
cache_dir = LLAMA_3P3_70B_MODEL_DIR

print(f"Loading model {model_name} from {cache_dir}")

In [ ]:
# Load the model from on-prem, so you don't need to login to HuggingFace
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)

In [ ]:
def load_model(model_name, cache_dir):
    """
    Load the model and tokenizer from the Hugging Face Hub. If the model is not
    found, it will be downloaded and cached in the specified directory.

    Args:
        model_name (str): The name of the model to load.
    """
    print(f"Loading model {model_name}...")
    model = AutoModel.from_pretrained(
        model_name,
        cache_dir=cache_dir,
        local_files_only=True
    )
    
    # Load tokenizer with same cache directory
    tokenizer = AutoModel.from_pretrained(
        model_name,
        cache_dir=cache_dir,
        local_files_only=True
    )

    return model, tokenizer

In [ ]:
load_model("meta-llama/Llama-3.3-70B-Instruct", cache_dir=LLAMA_3P3_70B_MODEL_DIR)

In [ ]:
def load_model(model_name):
    """
    Load the model and tokenizer from the Hugging Face Hub. If the model is not
    found, it will be downloaded and cached in the specified directory.

    Args:
        model_name (str): The name of the model to load.
    """
    print(f"Loading model {model_name}...")
    model = AutoModel.from_pretrained(model_name, cache_dir=LLAMA_3P3_70B_MODEL_DIR)
    tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=LLAMA_3P3_70B_MODEL_DIR)

    return model, tokenizer


def get_embeddings(model, tokenizer, text):
    """
    Get the embeddings for a given text using the specified model and tokenizer.
    Args:
        text (str): The input text to encode.
        tokenizer: The tokenizer to use for encoding the text.
        model: The model to use for generating embeddings.

    Returns:
        torch.Tensor: The embeddings for the input text.
    """
    # Tokenize with padding/truncation
    inputs = tokenizer(
        texts,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    
    # Get hidden states (batch_size x seq_len x hidden_dim)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state
    return embeddings

In [ ]:
print(LLAMA_3P3_70B_MODEL_DIR)

In [ ]:
model_name = "meta-llama/Llama-3.3-70B-Instruct"
model, tokenizer = load_model(model_name)

In [ ]:
texts = ["Hello, how are you?"]
embeddings = get_embeddings(model, tokenizer, texts)
print(embeddings)

In [ ]:
print(embeddings[0].shape)